## Import Libraries

In [1]:
from tqdm import tqdm
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import cv2
import timm
import torch
import torch.autograd.profiler as profiler

We need to measure three things potentially 4
- Model Size
- Inference
- Memory Usage

## MODELS

In [34]:
from torchvision.models import shufflenet_v2_x1_0
shufflenet = shufflenet_v2_x1_0(pretrained=True)
shufflenet = model = torch.load("./ShuffleNetModels/shuffle_trained_Deepfakes.pth", weights_only=False, map_location="cpu")

In [36]:
mobilenet = timm.create_model('mobilenetv2_100', pretrained=True, num_classes=1)
mobilenet = torch.load("./MobileNetModels/mobilenet_trained_Deepfakes.pth", weights_only=False, map_location="cpu")

In [41]:
ghostnet = torch.load("./GhostNetModels/ghostnet_trained_Deepfakes.pth", weights_only=False, map_location="cpu")

In [42]:
efficientnet = torch.load("./EfficientNetLiteModels/efficientnet_trained_Deepfakes.pth", weights_only=False, map_location="cpu")

In [44]:
tinyvit = torch.load("./TinyVitModels/tinyvit_trained_Deepfakes.pth", weights_only=False, map_location="cpu")

In [46]:
mobilevit = torch.load("./MobileViTModels/mobilevit_trained_Deepfakes.pth", weights_only=False, map_location="cpu")

In [47]:
convnext = torch.load("./ConvNeXtModels/convnext_trained_FaceShifter.pth", weights_only=False, map_location="cpu")

# Model Size

In [24]:
def get_model_size(model):
    """
    Returns the size of a PyTorch model in megabytes.

    Args:
        model (torch.nn.Module): The PyTorch model.

    Returns:
        float: The size of the model in megabytes.
    """

    # Save the model to a temporary file
    torch.save(model.state_dict(), "temp_model.pth")

    # Get the file size in bytes
    size_bytes = os.path.getsize("temp_model.pth")

    # Delete the temporary file
    os.remove("temp_model.pth")

    # Convert bytes to megabytes
    size_mb = size_bytes / (1024 * 1024)
    print(f"Model size: {size_mb:.2f} MB")
    return size_mb


# Inference

In [ ]:
import torch
import cv2
import time

def measure_inference_time(model, frame_path, num_iterations=1000, device="cpu"):
    """
    Measures the average inference time of a model on a single input frame.

    Args:
        model (torch.nn.Module): The deep learning model to evaluate.
        frame_path (str): Path to the image file used for inference.
        num_iterations (int, optional): Number of times to repeat inference for accurate timing. Default is 1000.
        device (str, optional): Device to run inference on ("cpu" or "cuda"). Default is "cpu".

    Returns:
        tuple: (average inference time in milliseconds, standard deviation in milliseconds)
    """

    # Load the image from the given file path and convert it to a PyTorch tensor
    # - `cv2.imread()` loads the image as a NumPy array (H x W x C, BGR format)
    # - `torch.from_numpy()` converts it to a PyTorch tensor
    # - `.permute(2,0,1)` changes the shape to (C x H x W) for PyTorch compatibility
    # - `.float()` ensures it's a floating-point tensor
    frame = torch.from_numpy(cv2.imread(frame_path)).permute(2,0,1).float()

    # Move model to the specified device (CPU or GPU)
    model.to(device)

    # Set the model to evaluation mode (disables dropout, batch norm updates)
    model.eval()

    # Add a batch dimension to the input tensor (B x C x H x W) if needed
    frame = frame.unsqueeze(0)  

    # Move input frame to the specified device
    frame = frame.to(device)

    # List to store inference times for each iteration
    inference_times = []

    # Disable gradient computation for efficiency during inference
    with torch.no_grad():
        for _ in range(num_iterations):
            start_time = time.time()  # Record start time
            _ = model(frame)  # Run inference (output ignored)
            end_time = time.time()  # Record end time

            # Compute and store inference time for this iteration
            inference_times.append(end_time - start_time)

    # Compute the average inference time in milliseconds
    avg_inference_time = sum(inference_times) / len(inference_times) * 1000

    # Compute the standard deviation of inference time (converted to milliseconds)
    std_inference_time = torch.std(torch.tensor(inference_times)).item() * 1000

    # Print results
    print(f"Average inference time: {avg_inference_time:.2f} milliseconds")
    print(f"Standard deviation: {std_inference_time:.2f} milliseconds")

    # Return the average inference time and standard deviation in milliseconds
    return avg_inference_time, std_inference_time


# Memory Usage

In [77]:
def track_cpu_memory(model, input_shape=(1, 3, 224, 224)):
    """
    Measures peak CPU memory usage during inference using PyTorch Profiler.

    Args:
        model (torch.nn.Module): The model to profile.
        input_shape (tuple): Shape of the input tensor.

    Returns:
        float: Peak memory allocated (in MB)
    """
    device = "cpu"  # Ensure model runs on CPU

    # Move model to CPU
    model.to(device)
    model.eval()  # Ensure model is in evaluation mode

    # Create a dummy input tensor on CPU
    input_tensor = torch.randn(input_shape).to(device)

    # Profile CPU memory usage
    with torch.no_grad():
        with profiler.profile(profile_memory=True) as prof:
            model(input_tensor)

    # Extract peak CPU memory allocated
    peak_memory_mb = max(event.self_cpu_memory_usage for event in prof.key_averages()) / 1024**2

    print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))
    print(f"\n🔹 CPU Peak Memory Allocated: {peak_memory_mb:.2f} MB")

    return peak_memory_mb



In [89]:
track_cpu_memory(convnext, input_shape=(1, 3, 224, 224))

----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                        Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 aten::addmm        65.83%     140.040ms        68.28%     145.260ms       3.926ms      40.91 Mb      40.91 Mb            37  
                  aten::gelu         8.44%      17.955ms         8.44%      17.955ms     997.491us      32.73 Mb      32.73 Mb            18  
                 aten::empty         0.46%     983.540us         0.46%     983.540us       8.265us      26.81 Mb      26.81 Mb           119  
                   aten::add         2.75%       5.844ms         2.77%       5.899ms     226.893us      11.37 Mb      11.37 Mb            26  

40.913089752197266